In [9]:
import os
import subprocess
import numpy as np
import pandas as pd
import seaborn as sns

In [11]:
def find_files(directory):
    for root, _, files in os.walk(directory):
        for file in files:
            yield os.path.join(root, file)
            
            
directory_to_search = '../partial_data_couchbase_512KiB/'
# directory_to_search = '/home/gpu2/sunt/Docker_dedup/data/Docker/uncompressed_data_couchbase_512KiB/'
all_files = list(find_files(directory_to_search))

print(f'chunk number: {len(all_files)}')

chunk number: 73406


In [12]:
print(all_files[0])

../partial_data_couchbase_512KiB/sha256_5ef8813a4118b27310b9f2ae576098f0466002f73051bcd0c41d87881230456c_276_block_683.bin


# 1. LSH-based MLE

## Multiprocess to calculate

In [13]:
import os
import subprocess
import numpy as np
import pandas as pd
import seaborn as sns

import subprocess
import pandas as pd
from multiprocessing import Pool

def find_files(directory):
    for root, _, files in os.walk(directory):
        for file in files:
            yield os.path.join(root, file)


# directory_to_search = './uncompressed_data_ubuntu_4KiB/'
# all_files = list(find_files(directory_to_search))

print(f'chunk number: {len(all_files)}')


def process_file(file):
    status, output = subprocess.getstatusoutput('./finesse ' + file + ' 48 3 12')
    output = output.split('\n')
    h = output[0].split(': ')[-1]
    sf1 = output[1]
    sf2 = output[2]
    sf3 = output[3]
    return file, h, sf1, sf2, sf3

# def process_main():
all_files = list(find_files(directory_to_search))  
print(f"Total files: {len(all_files)}")
pool_size = 30  # Adjust based on your system capability

sfs = []
with Pool(pool_size) as p:
    for i, result in enumerate(p.imap(process_file, all_files), 1):
        sfs.append(result)
        if i % 10000 == 0:
            print(f"Processed {i}/{len(all_files)} files")

print('\nFinish parallel computing')
sfs_pd = pd.DataFrame(sfs, columns=['file name', 'h', 'sf1', 'sf2', 'sf3'])

# print(sfs_pd)

# sfs_pd.to_pickle('./sfs_pd_partial_data_pytorch_512KiB.pkl')

# process_main()

chunk number: 73406
Total files: 73406
Processed 10000/73406 files
Processed 20000/73406 files
Processed 30000/73406 files
Processed 40000/73406 files
Processed 50000/73406 files
Processed 60000/73406 files
Processed 70000/73406 files

Finish parallel computing


In [16]:
sfs_pd.to_pickle('./sfs_pd(partial_data_ubuntu).pkl')

In [14]:
sfs_pd.describe()

,file name,h,sf1,sf2,sf3
count,73406,73406,73406,73406,73406
unique,73406,29059,26935,26509,24926
top,../partial_data_couchbase_512KiB/sha256_40dd5b...,-3847956920104831106,-8712679140581332489,-8712679140581332489,-8712679140581332489
freq,1,3156,7373,7373,7374


In [15]:
sfs_pd['sf3'].value_counts()

sf3
-8712679140581332489    7374
4872234842726086240     3157
-1893084533538898953    2958
-3562642815723339238    2508
2101747852265831646     2445
                        ... 
-7589928164018896471       1
-7664904731149841311       1
-2510670744815703913       1
3796861271265640188        1
-8961892939105730193       1
Name: count, Length: 24926, dtype: int64

In [16]:

sf3_counts = sfs_pd['sf3'].value_counts()  

sf3_counts = sf3_counts - 1  
  
 
total = sf3_counts.sum()  
  
# Print the finesse results 
print("The total sum after subtracting 1 from each count is: ", total) 
print(total*512/1024/1024)   

The total sum after subtracting 1 from each count is:  48480
23.671875


# Use fastcdc for the finesse results

In [17]:
new_df = sfs_pd.groupby('sf3')['file name'].agg(lambda x: list(x)).reset_index()  
new_df.columns = ['sf3', 'file name'] 
new_df

,sf3,file name
0,-1000163073875948969,[../partial_data_couchbase_512KiB/sha256_85ae4...
1,-1000921660620857690,[../partial_data_couchbase_512KiB/sha256_56326...
2,-1001806519935284385,[../partial_data_couchbase_512KiB/sha256_191c1...
3,-1002677444869268416,[../partial_data_couchbase_512KiB/sha256_07696...
4,-1003827358512028569,[../partial_data_couchbase_512KiB/sha256_91aeb...
...,...,...
24921,998112322438096319,[../partial_data_couchbase_512KiB/sha256_d1cdc...
24922,998294922023521085,[../partial_data_couchbase_512KiB/sha256_d1cdc...
24923,998602743249504034,[../partial_data_couchbase_512KiB/sha256_69e6e...
24924,998629786024765294,[../partial_data_couchbase_512KiB/sha256_191c1...


In [18]:
new_df = new_df[new_df['file name'].apply(lambda x: len(x) > 1)]  
new_df = new_df.reset_index(drop=True)  
new_df

,sf3,file name
0,-1001806519935284385,[../partial_data_couchbase_512KiB/sha256_191c1...
1,-1002677444869268416,[../partial_data_couchbase_512KiB/sha256_07696...
2,-1003827358512028569,[../partial_data_couchbase_512KiB/sha256_91aeb...
3,-1004158725143750423,[../partial_data_couchbase_512KiB/sha256_df470...
4,-1004951072507015705,[../partial_data_couchbase_512KiB/sha256_91aeb...
...,...,...
14214,997133920672704700,[../partial_data_couchbase_512KiB/sha256_85ae4...
14215,998112322438096319,[../partial_data_couchbase_512KiB/sha256_d1cdc...
14216,998294922023521085,[../partial_data_couchbase_512KiB/sha256_d1cdc...
14217,998629786024765294,[../partial_data_couchbase_512KiB/sha256_191c1...


In [19]:
import multiprocessing
from fastcdc import fastcdc
from hashlib import sha256

num_process=30

def process_file(file_name):
    # status, output = subprocess.getstatusoutput('fastcdc -mi 256 -s 512 -ma 1024 ' + file_name)
    # output = output.split('\n')
    results = []
    output = list(fastcdc(file_name, 256, 512, 1024, fat=True, hf=sha256))
    for i in range(len(output)):
        results.append([file_name, output[i].hash, output[i].length])
    # for line in output:
    #     temp = line.split(' ')
    #     temp_hash = temp[0].split('=')[-1]
    #     temp_offset = temp[1].split('=')[-1]
    #     temp_size = temp[2].split('=')[-1]
    #     results.append([file_name, temp_hash, temp_size])
    return results

def get_reduce_ration(fastcdc_df):

    fastcdc_df['size'] = fastcdc_df['size'].astype(int)

    # Calculate the total size of each hash value
    hash_total_size = fastcdc_df.groupby('hash')['size'].sum()

    # Calculate the number of occurrences of each hash
    hash_counts = fastcdc_df['hash'].value_counts()
    hashes_more_than_once = hash_counts[hash_counts > 1].index
    duplicated_first_instances = fastcdc_df[fastcdc_df['hash'].isin(hashes_more_than_once)].drop_duplicates('hash')
    size_sum_duplicated_first = duplicated_first_instances['size'].sum()
    
    hashes_once_only = hash_counts[hash_counts == 1].index
    instances_once_only = fastcdc_df[fastcdc_df['hash'].isin(hashes_once_only)]
    size_sum_once_only = instances_once_only['size'].sum()
    
    # Calculate the redundant size: for each hash, its total size minus the size of the first instance
    reduced_size = (hash_total_size - fastcdc_df.drop_duplicates('hash').set_index('hash')['size']).sum()
    
    
    # Calculate the total size
    total_size = hash_total_size.sum()

    # print(f"reduce size is {reduced_size}")
    # print(f"all size is {total_size}")
    return reduced_size, total_size, size_sum_duplicated_first, size_sum_once_only

def get_cluster(cluster_id):
    fastcdc_list = []
    cluster_files = new_df.iloc[cluster_id].at['file name']

    with multiprocessing.Pool(processes=num_process) as pool:
        results = pool.map(process_file, cluster_files)
        for result in results:
            fastcdc_list.extend(result)

    fastcdc_df = pd.DataFrame(fastcdc_list, columns=['file name', 'hash', 'size'])
    return fastcdc_df


all_cluster_size = []
all_cluster_reducesize = []
all_cluster_sum_duplicated_first = []
all_cluster_sum_once_only = []

for cluster_id in range(new_df.shape[0]):
    # print(f"cluster:{cluster_id}")
    temp_df = get_cluster(cluster_id)
    temp_reduce, temp_all, temp_size_sum_duplicated_first, temp_size_sum_once_only = get_reduce_ration(temp_df)
    print(f"cluster:{cluster_id}, reduced size:{temp_reduce}, cluster total size: {temp_all}, duplicated first: {temp_size_sum_duplicated_first}, once_only: {temp_size_sum_once_only}")
    all_cluster_size.append(temp_all)
    all_cluster_reducesize.append(temp_reduce)
    all_cluster_sum_duplicated_first.append(temp_size_sum_duplicated_first)
    all_cluster_sum_once_only.append(temp_size_sum_once_only)

total = sum(all_cluster_size)
total_reduce = sum(all_cluster_reducesize)
total_duplicated_first = sum(all_cluster_sum_duplicated_first)
total_once_only = sum(all_cluster_sum_once_only)

print("{} {} {}".format(total_reduce, total, total_reduce * 1.0 / total))
print(f'total reduced: {total_reduce/1024/1024/1024}GB')
print(f'total: {total/1024/1024/1024}GB')
print(f'total duplicated first: {total_duplicated_first/1024/1024/1024}GB')
print(f'total once only: {total_once_only/1024/1024/1024}GB')

cluster:0, reduced size:524307, cluster total size: 1048614, duplicated first: 524307, once_only: 0
cluster:1, reduced size:524506, cluster total size: 1049012, duplicated first: 524506, once_only: 0
cluster:2, reduced size:524288, cluster total size: 1048576, duplicated first: 524288, once_only: 0
cluster:3, reduced size:524288, cluster total size: 1048576, duplicated first: 524288, once_only: 0
cluster:4, reduced size:593821, cluster total size: 1048576, duplicated first: 454755, once_only: 0
cluster:5, reduced size:524509, cluster total size: 1049018, duplicated first: 524509, once_only: 0
cluster:6, reduced size:3138560, cluster total size: 3145728, duplicated first: 7168, once_only: 0
cluster:7, reduced size:524288, cluster total size: 1048576, duplicated first: 524288, once_only: 0
cluster:8, reduced size:524400, cluster total size: 1048800, duplicated first: 524400, once_only: 0
cluster:9, reduced size:539195, cluster total size: 1048822, duplicated first: 509627, once_only: 0
c

# 2. Message-locked encryption (MLE)

In [20]:
import hashlib
def hash_file(filename):
    sha256_hash = hashlib.sha256()
    with open(filename, "rb") as f:
        for byte_block in iter(lambda: f.read(4096), b""):
            sha256_hash.update(byte_block)
    return sha256_hash.hexdigest()

sha256_list = [[], []]
cnt = 0
for file in all_files:
    # status, output = subprocess.getstatusoutput('sha256sum ' + file)
    output = hash_file(file)
    cnt += 1
    print(cnt, end='\r')
    # output = output.split('  ')
    sha256_list[0].append(file)
    sha256_list[1].append(output)

ssha256_pd=pd.DataFrame(sha256_list)
ssha256_pd = ssha256_pd.T
ssha256_pd.rename(columns={0:'file name', 1:'sha256'},inplace=True)
ssha256_pd

,file name,sha256
0,../partial_data_couchbase_512KiB/sha256_5ef881...,2de36f1edcb1c2f394719bc2e12c6730112d12e01aea42...
1,../partial_data_couchbase_512KiB/sha256_e8d510...,f93611298f2341e9035ccf37dd546b921fa0ed7d9db746...
2,../partial_data_couchbase_512KiB/sha256_c03232...,3d2cd576a881178b85f36aecb8f57915adb51a493142dc...
3,../partial_data_couchbase_512KiB/sha256_e8d510...,8a7f4ccae3fca81fb4412cf793199af231c2f22569ce63...
4,../partial_data_couchbase_512KiB/sha256_4b2c5f...,295404099ca190d3fe607fb780ddc46f28c96080cad93e...
...,...,...
73401,../partial_data_couchbase_512KiB/sha256_91aeb9...,082cec5267c9be635bfb7ec7693033a71a3b77b75c8822...
73402,../partial_data_couchbase_512KiB/sha256_9a6ce3...,95a3c876bc8674dc66b5555bc3e00cd51afb3cada354c4...
73403,../partial_data_couchbase_512KiB/sha256_0a161b...,e76ca609418d07f24e20b8b92069cf00c2efa7f1fb2fd5...
73404,../partial_data_couchbase_512KiB/sha256_fb369c...,c69077dbf4640fa9bf2db0f11459121d7e37c3ead799ff...


In [21]:
ssha256_pd.describe()

,file name,sha256
count,73406,73406
unique,73406,29059
top,../partial_data_couchbase_512KiB/sha256_40dd5b...,95a3c876bc8674dc66b5555bc3e00cd51afb3cada354c4...
freq,1,3156


In [22]:
ssha256_pd['sha256'].value_counts()

sha256
95a3c876bc8674dc66b5555bc3e00cd51afb3cada354c477d63a0a7ac30c5bdc    3156
36e095a1675499f7dccd0c2ff1117bf2e9673a86f310c4a219e07d22c832f9d9    2956
b83b202d20563078043cf2d6d7a1b5294fc9fa35ea76df0692978ecf95f3d4bf    2926
267c9c18367967ba9c2040a5850947d4cb1e432f5bd9a94002d198459e4a237b    2506
e76ca609418d07f24e20b8b92069cf00c2efa7f1fb2fd5efac82215b02ae999b    2443
                                                                    ... 
a6709dc3b5310ebfbba593d145173f6e0be2ab43cf105d0f8c4c551bdb2b5e5c       1
7afc06b555a3a0a182bc3181fc21507dddb7609d5ed632bcfb3b035fa34aa4e9       1
9390b3755d2be17a238fc5bc8c65052ce0df82efa69c10727cd197864f63feaa       1
cc51caca2da8d19c781d5684fa58fdbadf66d2af0009355d284bd9231175bffe       1
15a470f62e24a413f665960c318ede259e53c27e6d34dbce1a2aeb144a8f2fe7       1
Name: count, Length: 29059, dtype: int64

In [23]:
# Group by 'sha256' and calculate the total size of the duplicated files  
ssha256_pd['file size'] = ssha256_pd['file name'].apply(os.path.getsize)  
grouped = ssha256_pd.groupby('sha256').apply(lambda x: (len(x) - 1) * x['file size'].iloc[0] if len(x) > 1 else 0)  
  
# Calculate the total size of all the duplicated files  
total_deduplicated_size = grouped.sum()  
print(f'{total_deduplicated_size/1024/1024/1024} GiB,  {total_deduplicated_size}')
    

21.65380859375 GiB,  23250599936


In [24]:
import os
import pandas as pd

ssha256_pd['file size'] = ssha256_pd['file name'].apply(os.path.getsize)

first_file_sizes = ssha256_pd.groupby('sha256').apply(lambda x: x['file size'].iloc[0] if len(x) > 1 else 0)
total_first_file_sizes = first_file_sizes.sum()

total_file_size = ssha256_pd['file size'].sum()
unique_files = ssha256_pd.groupby('sha256').filter(lambda x: len(x) == 1)
total_unique_file_size = unique_files['file size'].sum()


print(f'Total size of files with a unique sha256 hash: {total_unique_file_size/1024/1024/1024} GiB, {total_unique_file_size} bytes')

print(f'Total size of the first files in each duplicate group: {total_first_file_sizes/1024/1024/1024} GiB, {total_first_file_sizes} bytes')
print(f'Total size of all files: {total_file_size/1024/1024/1024} GiB, {total_file_size} bytes')


Total size of files with a unique sha256 hash: 7.3056640625 GiB, 7844397056 bytes
Total size of the first files in each duplicate group: 6.88330078125 GiB, 7390887936 bytes
Total size of all files: 35.8427734375 GiB, 38485884928 bytes
